In [1]:
from face_extractor import Extractor

In [2]:
Extractor.extract(imagePath='test/Marilyn_Monroe.jpg',cascadeFile=Extractor.HAARCASCADE_ALT2,size=30)

array([[ 449,  544, 1349, 1349]], dtype=int32)

In [3]:
from keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using CNTK backend


In [4]:
def create_model(size):
    model = Sequential()
    init = 'he_normal'
    model.add(Conv2D(32, (2, 2), activation = 'relu', name = 'block1_conv1', 
                            kernel_initializer = init, input_shape = (size, size, 3)))
    model.add(Conv2D(32, (2, 2), activation = 'relu', name = 'block1_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((2, 2), name = 'block1_pool'))

    model.add(Conv2D(64, (2, 2), activation = 'relu', name = 'block2_conv1', kernel_initializer = init))
    model.add(Conv2D(128, (2, 2), activation = 'relu', name = 'block2_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((2, 2), name = 'block2_pool'))

    model.add(Conv2D(256, (2, 2), activation = 'relu', name = 'block3_conv1', kernel_initializer = init))
    model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block3_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((2, 2), name = 'block3_pool'))
    
#     model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block4_conv1', kernel_initializer=init))
#     model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block4_conv2', kernel_initializer=init))
#     model.add(MaxPooling2D((2, 2), name = 'block4_pool'))
    
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu', kernel_initializer = init, name = 'fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'relu', kernel_initializer = init, name = 'fc2'))
    model.add(Dropout(0.5))
    model.add(Dense(88, activation = 'softmax', name = 'predictions'))
    
    optim = Adam(lr = 0.0005)
    
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    return model

model = create_model(size = 30)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 29, 29, 32)        416       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 28, 28, 32)        4128      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 14, 14, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 13, 13, 64)        8256      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 12, 12, 128)       32896     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 6, 6, 128)         0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 5, 5, 256)         131328    
__________

In [5]:
model.load_weights(filepath='model.h5')

In [6]:
Marilyn_Monroe = img_to_array(load_img('test/Marilyn_Monroe_0.jpg'))
Marilyn_Monroe = Marilyn_Monroe.reshape((1,) + Marilyn_Monroe.shape) 

In [7]:
model.predict_classes(Marilyn_Monroe)

1/1 [==============================] - 0s


array([65], dtype=int64)

In [8]:
import os
sorted(os.listdir('data/train'))[65]

'marilyn monroe'

In [9]:
with open("classes.txt", "wb") as f:
    for cel in os.listdir('data/train'):
        f.write((cel+'\n').encode('utf8'))
